In [6]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [8]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/03 21:07:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/03 21:07:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/03/03 21:07:31 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
spark.version

'3.5.1'

In [9]:
!ls -lh fhv_tripdata_2019-08.csv

-rw-r--r-- 1 root root 114M Mar  3 21:00 fhv_tripdata_2019-08.csv


In [11]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropOff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
      types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [16]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhv_tripdata_2019-10.csv')

df = df.repartition(6)

df.write.parquet('data/pq/fhv/2019/10/', compression=None)

In [22]:
df = spark.read.parquet('data/pq/fhv/2019/10/')
import sys

# Convert DataFrame to RDD and then apply the same logic as above
rdd = df.rdd

# Function to estimate the size of each partition in MB
def get_partition_size_in_mb(iterator):
    size_bytes = sum(sys.getsizeof(elem) for elem in iterator)
    size_mb = size_bytes / (1024 * 1024)  # Convert bytes to megabytes
    yield size_mb

# Apply the function to each partition of the RDD
partition_sizes_mb = rdd.mapPartitions(get_partition_size_in_mb).collect()

# Print the size of each partition in MB
for i, size in enumerate(partition_sizes_mb):
    print(f"Partition {i}: {size} MB")

Partition 0: 31.366249084472656 MB
Partition 1: 31.366348266601562 MB
Partition 2: 31.366249084472656 MB
Partition 3: 31.36614990234375 MB
Partition 4: 31.366249084472656 MB
Partition 5: 31.36614990234375 MB


**Q3**: How many taxi trips were there on February 15?

In [23]:
from pyspark.sql import functions as F

In [26]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .filter("pickup_date = '2019-10-15'") \
    .count()

62610

In [27]:
df.registerTempTable('fhv_2019_10')

/opt/conda/lib/python3.8/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [28]:
spark.sql("""
SELECT
    COUNT(1)
FROM 
    fhv_2019_10
WHERE
    to_date(pickup_datetime) = '2019-10-15';
""").show()

+--------+
|count(1)|
+--------+
|   62610|
+--------+



**Q4**: Longest trip for each day

In [29]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropOff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [31]:
df \
    .withColumn('duration', df.dropOff_datetime.cast('long') - df.pickup_datetime.cast('long')) \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .groupBy('pickup_date') \
        .max('duration') \
    .orderBy('max(duration)', ascending=False) \
    .limit(5) \
    .show()

+-----------+-------------+
|pickup_date|max(duration)|
+-----------+-------------+
| 2019-10-28|   2272149000|
| 2019-10-11|   2272149000|
| 2019-10-31|    315620787|
| 2019-10-01|    252460901|
| 2019-10-17|     31658400|
+-----------+-------------+



In [34]:
spark.sql("""
SELECT
    to_date(pickup_datetime) AS pickup_date,
    MAX((CAST(dropoff_datetime AS LONG) - CAST(pickup_datetime AS LONG)) / 60 / 60) AS duration
FROM 
    fhv_2019_10
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 10;
""").show()

+-----------+------------------+
|pickup_date|          duration|
+-----------+------------------+
| 2019-10-28|          631152.5|
| 2019-10-11|          631152.5|
| 2019-10-31| 87672.44083333334|
| 2019-10-01| 70128.02805555557|
| 2019-10-17|            8794.0|
| 2019-10-26| 8784.166666666666|
| 2019-10-30|1464.5344444444445|
| 2019-10-25|1056.8266666666666|
| 2019-10-02| 769.2313888888889|
| 2019-10-23| 745.6166666666667|
+-----------+------------------+



**Q5**: Most frequent `dispatching_base_num`

How many stages this spark job has?



In [36]:
spark.sql("""
SELECT
    dispatching_base_num,
    COUNT(1)
FROM 
    fhv_2019_10
GROUP BY
    1
ORDER BY
    2 ASc
LIMIT 5;
""").show()

+--------------------+--------+
|dispatching_base_num|count(1)|
+--------------------+--------+
|              B02898|       1|
|              B02956|       1|
|              B02760|       1|
|              b00706|       1|
|             B02004 |       2|
+--------------------+--------+



In [46]:
df \
    .groupBy('dispatching_base_num') \
        .count() \
    .orderBy('count', ascending=False) \
    .limit(5) \
    .show()

+--------------------+-------+
|dispatching_base_num|  count|
+--------------------+-------+
|              B02510|3233664|
|              B02764| 965568|
|              B02872| 882689|
|              B02875| 685390|
|              B02765| 559768|
+--------------------+-------+



**Q6**: Most common locations pair

In [37]:
df_zones = spark.read.parquet('zones')

In [38]:
df_zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [39]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropOff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [40]:
df_zones.registerTempTable('zones')

In [42]:
spark.sql("""
SELECT
    CONCAT(pul.Zone, ' / ', dol.Zone) AS pu_do_pair,
    COUNT(1)
FROM 
    fhv_2019_10 fhv LEFT JOIN zones pul ON fhv.PULocationID = pul.LocationID
                      LEFT JOIN zones dol ON fhv.DOLocationID = dol.LocationID
GROUP BY 
    1
ORDER BY
    2 ASC
LIMIT 5;
""").show()

+--------------------+--------+
|          pu_do_pair|count(1)|
+--------------------+--------+
|Flushing / Prospe...|       1|
|Parkchester / Upp...|       1|
|Stuyvesant Height...|       1|
|Sunset Park West ...|       1|
|Co-Op City / Two ...|       1|
+--------------------+--------+



In [45]:
spark.sql("""
SELECT
    pul.Zone as pickupZone,
    COUNT(1)
FROM 
    fhv_2019_10 fhv LEFT JOIN zones pul ON fhv.PULocationID = pul.LocationID
                      LEFT JOIN zones dol ON fhv.DOLocationID = dol.LocationID
GROUP BY 
    1
ORDER BY
    2 ASC
LIMIT 5;
""").show()

+--------------------+--------+
|          pickupZone|count(1)|
+--------------------+--------+
|         Jamaica Bay|       1|
|Governor's Island...|       2|
| Green-Wood Cemetery|       5|
|       Broad Channel|       8|
|     Highbridge Park|      14|
+--------------------+--------+

